#  Homework week 9
## MBIO612
### Gabrielle Martineau

In [3]:
#Set working directory
setwd("~/U_of_H/FALL_2021/MBIO612/Week_9")

#download necessary libraries
library(tidyverse)
library(readr)
library(dplyr)
library(stats)
library(MASS)

In [4]:
#load data
survey_data<- read_delim("RIKZ.txt", delim = "\t", 
                          escape_double = FALSE, trim_ws = TRUE)
is_tibble(survey_data)


-- Column specification --------------------------------------------------------
cols(
  .default = col_double()
)
i Use `spec()` for the full column specifications.



[1] TRUE

In [5]:
#display the first 6 lines of the tables
head(survey_data)

Sample,C1,P1,P2,P3,P4,P5,P6,P7,P8,...,exposure,salinity,temperature,NAP,penetrability,grainsize,humus,chalk,sorting1,Beach
1,4,0,0,0,0,0,1,0,0,...,10,29.4,17.5,0.045,253.9,222.5,0.05,2.05,69.830,1
2,0,0,1,0,0,0,0,0,0,...,10,29.4,17.5,-1.036,226.9,200.0,0.30,2.50,59.000,1
3,0,0,3,0,0,0,0,0,0,...,10,29.4,17.5,-1.336,237.1,194.5,0.10,3.45,59.220,1
4,0,0,0,0,0,0,0,0,0,...,10,29.4,17.5,0.616,248.6,221.0,0.15,1.60,67.750,1
5,1,0,0,0,0,0,0,0,0,...,10,29.4,17.5,-0.684,251.9,202.0,0.05,2.45,57.760,1
6,0,0,0,0,0,1,0,0,0,...,8,29.6,20.8,1.190,250.1,192.5,0.10,2.50,53.075,2


In [6]:
#create a subset of the the variables that are compiled for calculation of diversity
survey_data_diversity= survey_data[,2:76]

#Replace the values with 0 or 1
survey_data_diversity[survey_data_diversity > 0]<-1
survey_data_diversity[survey_data_diversity == 0]<-0

#sum by row and compute diversity per sample
diversity = apply(survey_data_diversity,1,sum)

#Add the column diversity to the tibble
survey_data<- mutate(survey_data, diversity_per_sample=diversity)

In [7]:
#Create a copy of the variable survey_data that does not have columns C1 P1-P25, N1, CR1-28, M1-17 and I1-5. Call this variable survey_data_richness
drop <- names(survey_data_diversity)
survey_data_richness = survey_data[,!(names(survey_data) %in% drop)]

In [13]:
#create model with given predictors
model_richness = lm(diversity_per_sample ~ angle1 + angle2+ exposure + salinity + temperature +NAP +penetrability +grainsize +humus +chalk + sorting1+ Beach, data= survey_data_richness)
summary(model_richness)
range(survey_data_richness$diversity_per_sample)


Call:
lm(formula = diversity_per_sample ~ angle1 + angle2 + exposure + 
    salinity + temperature + NAP + penetrability + grainsize + 
    humus + chalk + sorting1 + Beach, data = survey_data_richness)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.4344 -1.3933 -0.5195  0.7726 11.6044 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -43.371160  47.580750  -0.912    0.369    
angle1         -0.009397   0.010924  -0.860    0.396    
angle2          0.053760   0.040977   1.312    0.199    
exposure       -0.541853   1.856944  -0.292    0.772    
salinity        1.020630   1.311337   0.778    0.442    
temperature     1.195516   1.230227   0.972    0.338    
NAP            -2.765120   0.580865  -4.760 3.98e-05 ***
penetrability  -0.006443   0.007021  -0.918    0.366    
grainsize       0.019833   0.030860   0.643    0.525    
humus           0.306015   9.087738   0.034    0.973    
chalk          -0.137526   0.138167  -0.995    0.327    
sor

[1]  0 22

There is an Intercept, and coefficients for each predictor. The coefficients quantify how much each predictor vary in richness relative to the intercept. The sign of the coefficient indicates if the variation in richness is positive or negative. Therefore, a coefficient estimate of 1.19 for temperature means that species richness increases (away from zero) as the temperature increases, at a rate 0f 1.19 per degree or unit of temperature. The p-value indicates if the effect of the predictor is significant and it seems to be the case only with the NAP predictor (p<0.05).

The adjusted r-square indicates that nearly 62% of the variation in the response variable can be explained by the multiple regression performed here. This value is adjusted for the high number of predictor. This result is high but not that high, considering the high number of predictors included in the analysis.

In addition, the residual standard error indicates that we are making predictions that have, on average, an error of 3.1. This is a lot considering that the diversity per sample ranges from 0 to 22. 

In [12]:
#Build a model that includes all parameters
model_richness_all_parameters = lm(diversity_per_sample ~ week+ angle1 + angle2+ exposure + salinity + temperature +NAP +penetrability +grainsize +humus +chalk + sorting1+ Beach, data= survey_data)
summary(model_richness_all_parameters)


Call:
lm(formula = diversity_per_sample ~ week + angle1 + angle2 + 
    exposure + salinity + temperature + NAP + penetrability + 
    grainsize + humus + chalk + sorting1 + Beach, data = survey_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.5434 -1.3910 -0.2209  0.8268 11.6548 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)   -21.714257  62.877690  -0.345    0.732    
week            1.537678   2.873627   0.535    0.596    
angle1         -0.008529   0.011166  -0.764    0.451    
angle2          0.078737   0.062419   1.261    0.217    
exposure       -0.192513   1.988243  -0.097    0.923    
salinity       -0.149684   2.557775  -0.059    0.954    
temperature     1.645499   1.501715   1.096    0.282    
NAP            -2.727377   0.591672  -4.610 6.55e-05 ***
penetrability  -0.008968   0.008525  -1.052    0.301    
grainsize       0.015923   0.032054   0.497    0.623    
humus          -0.361549   9.275088  -0.039    0.969    
chalk

The model that includes all predictors yields similar results than the previous model. The adjusted R square is slighlty lower when accounting for weeks, and the coefficients are slightly different

In [15]:
#compare the models by their AIC
stepAIC(model_richness_all_parameters)

Start:  AIC=114.2
diversity_per_sample ~ week + angle1 + angle2 + exposure + salinity + 
    temperature + NAP + penetrability + grainsize + humus + chalk + 
    sorting1 + Beach

                Df Sum of Sq    RSS    AIC
- humus          1     0.015 305.62 112.21
- salinity       1     0.034 305.64 112.21
- exposure       1     0.092 305.70 112.22
- sorting1       1     0.106 305.72 112.22
- grainsize      1     2.433 308.04 112.56
- week           1     2.823 308.43 112.62
- Beach          1     3.646 309.25 112.74
- angle1         1     5.751 311.36 113.04
- chalk          1     7.990 313.60 113.36
- penetrability  1    10.909 316.52 113.78
- temperature    1    11.837 317.45 113.91
<none>                       305.61 114.20
- angle2         1    15.686 321.30 114.46
- NAP            1   209.475 515.08 135.69

Step:  AIC=112.21
diversity_per_sample ~ week + angle1 + angle2 + exposure + salinity + 
    temperature + NAP + penetrability + grainsize + chalk + sorting1 + 
    Beach

  


Call:
lm(formula = diversity_per_sample ~ temperature + NAP + chalk + 
    Beach, data = survey_data)

Coefficients:
(Intercept)  temperature          NAP        chalk        Beach  
   -15.9843       1.4742      -2.9506      -0.1336      -0.7831  


In this case, the model that best has the lowest AIC is the one that best fits our data. In this case, the best model includes the predictors temperature + NAP + chalk + Beach (AIC of 102). The algorithm does marginal testing by comparing the full model to the same models lacking one of the predictors and repeats that process, till it finds the best fit. 

However, the high amount of comparisons can introduce a multiple comparisons problem in this case, which lowers the power of our analysis and the certainty of the model selection

In [18]:
best_model_interaction =lm (diversity_per_sample ~ temperature + NAP + penetrability + chalk + 
    Beach+ temperature*NAP+ temperature*penetrability+ temperature*chalk + temperature*Beach + NAP*penetrability+ NAP*chalk + NAP*Beach + penetrability*chalk+ penetrability*Beach+ chalk*Beach, data=survey_data)
summary(best_model_interaction)


Call:
lm(formula = diversity_per_sample ~ temperature + NAP + penetrability + 
    chalk + Beach + temperature * NAP + temperature * penetrability + 
    temperature * chalk + temperature * Beach + NAP * penetrability + 
    NAP * chalk + NAP * Beach + penetrability * chalk + penetrability * 
    Beach + chalk * Beach, data = survey_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.8890 -1.1828 -0.4924  0.8587  9.1195 

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)  
(Intercept)               75.7748788 94.7984853   0.799    0.431  
temperature               -4.2620068  6.2318163  -0.684    0.499  
NAP                       10.4550588  7.1330821   1.466    0.153  
penetrability             -0.3362837  0.3406891  -0.987    0.332  
chalk                      2.1922209  3.4573983   0.634    0.531  
Beach                     -3.9347520  6.2441446  -0.630    0.534  
temperature:NAP           -0.8555739  0.4041905  -2.117    0.043 *
temperature:p

It seems like this model is the one that best fits our data. However, almost none of the predictors or their interactions seem to have a significant effect on the regression slope. It is hard to disentangle the contribution of each predictor to the variation of the response variable and there could be some collinearity between predictors. In this case, the Adjusted R-squared is of 0.64, which is a bit higher than the full model previously computed, confirming that the best model is able to explain a higher percentage of the variation in our data. The RSE is still around 3, which hasn't improved very much. 